## Homework 1 solution

Let's use the following convention for numbering legs:
```
 1--A--3
    |
    2
```

In [1]:
using TensorOperations
using LinearMaps

In [2]:
"""
    rand_UMPS(d, D; keep_it_real=true)

Return a random three-valent tensor A, that defines a uniform MPS (UMPS).
The bond dimension of the physical leg should be d, and the bond dimension
of the two "virtual" legs (the horizontal ones) should be D.
keep_it_real is keyword argument, for whether the matrix should be real or
complex.

This means you can call
`rand_UMPS(2, 9)`
or
`rand_UMPS(2, 9; keep_it_real=true)`
and they both give a you real A, but you can also call
`rand_UMPS(2, 9; keep_it_real=false)`
to get a complex A.
"""
function rand_UMPS(d, D; keep_it_real=true)
    shp = (D, d, D)
    if keep_it_real
        A = randn(shp)
    else
        A_real = randn(shp)
        A_imag = randn(shp)
        A = complex.(A_real, A_imag) / sqrt(2)
    end
    return A
end

rand_UMPS

In [3]:
"""
    tm(A)

Return the transfer matrix of A:
 --A---
   |  
 --A*--
"""
function tm(A)
    @tensor T[i1,i2,j1,j2] := A[i1,p,j1]*conj(A)[i2,p,j2]
end

tm

In [4]:
function eig_and_trunc(T, nev; by=identity, rev=false)
    S, U = eig(T)
    perm = sortperm(S; by=by, rev=rev)
    S = S[perm]
    U = U[:, perm]
    S = S[1:nev]
    U = U[:, 1:nev]
    return S, U
end

"""
    tm_eigs(A, dirn, nev)

Return some of the eigenvalues and vectors of the transfer matrix of A.
dirn should be "L", "R" or "BOTH", and determines which eigenvectors to return.
nev is the number of eigenpairs to return (starting with the eigenvalues with
largest magnitude).
"""
function tm_eigs_dense(A, dirn, nev)
    T = tm(A)
    D = size(T, 1)
    T = reshape(T, (D^2, D^2))
    nev = min(nev, D^2)
    
    result = ()
    if dirn == "R" || dirn == "BOTH"
        SR, UR = eig_and_trunc(T, nev; by=abs, rev=true)
        UR = [reshape(UR[:,i], (D, D)) for i in 1:nev]
        result = tuple(result..., SR, UR)
    end
    if dirn == "L" || dirn == "BOTH"
        SL, UL = eig_and_trunc(T', nev; by=abs, rev=true)
        UL = [reshape(UL[:,i], (D, D)) for i in 1:nev]
        result = tuple(result..., SL, UL)
    end
    return result
end

tm_eigs_dense

In [5]:
"""
    tm_l(A, x)

Return y, where
/------   /------A--
|       = |      |  
\- y* -   \- x* -A*-
"""
function tm_l(A, x)
    @tensor y[i, j] := (x[a, b] * A[b, p, j]) * conj(A[a, p, i])
    return y
end


"""
    tm_r(A, x)

Return y, where
-- y -\   --A-- x -\
      | =   |      |
------/   --A*-----/
"""
function tm_r(A, x)
    @tensor y[i, j] := A[i, p, a] * (conj(A[j, p, b]) * x[a, b])
    return y
end


tm_r

In [6]:
function tm_eigs_sparse(A, dirn, nev)
    if dirn == "BOTH"
        SR, UR = tm_eigs_sparse(A, "R", nev)
        SL, UL = tm_eigs_sparse(A, "L", nev)
        return SR, UR, SL, UL
    else
        D = size(A, 1)
        x = zeros(eltype(A), (D, D))
        if dirn == "L"
            f = v -> vec(tm_l(A, copy!(x, v)))
        else
            f = v -> vec(tm_r(A, copy!(x, v)))
        end

        fmap = LinearMap{eltype(A)}(f, D^2)
        S, U, nconv, niter, nmult, resid = eigs(fmap, nev=nev, which=:LM, ritzvec=true)
        U = [reshape(U[:,i], (D, D)) for i in 1:size(U, 2)]

        return S, U
    end
end

tm_eigs_sparse (generic function with 1 method)

In [7]:
function tm_eigs(A, dirn, nev; max_dense_D=10)
    D = size(A, 1)
    if D <= max_dense_D || nev >= D^2
        return tm_eigs_dense(A, dirn, nev)
    else
        return tm_eigs_sparse(A, dirn, nev)
    end
end

tm_eigs (generic function with 1 method)

In [8]:
"""
    normalize!(A)

Normalize the UMPS defined by A, and return the dominant left and right
eigenvectors l and r of its transfer matrix, normalized so that l'*r = 1.
"""
function normalize!(A)
    SR, UR, SL, UL = tm_eigs(A, "BOTH", 1)
    S1 = SR[1]
    A ./= sqrt(S1)
    
    l = UL[1]
    r = UR[1]  
    #We need this to be 1
    n = vec(l)'*vec(r)
    abs_n = abs(n)
    phase_n = abs_n/n
    sfac = 1.0/sqrt(abs_n)
    l .*= sfac/phase_n
    r .*= sfac
    return l, r
end

normalize!

## Homework 2 solution

In [9]:
"""
    tm_l_op(A, O, x)

Return y, where
/------   /------A--
|         |      |  
|       = |      O  
|         |      |  
\- y* -   \- x* -A*-
"""
function tm_l_op(A, O, x)
    @tensor y[i, j] := (x[a, b] * A[b, p2, j]) * (conj(A[a, p1, i]) * conj(O[p1, p2]))
    return y
end


"""
    tm_r_op(A, O, x)

Return y, where
-- y -\   --A-- x -\
      |     |      |
      | =   O      |
      |     |      |
------/   --A*-----/
"""
function tm_r_op(A, O, x)
    @tensor y[i, j] := (A[i, p1, a] * O[p1, p2]) * (conj(A[j, p2, b]) * x[a, b])
    return y
end


tm_r_op

In [10]:
"""
    expect_local(A, O, l, r)

Return the expectation value of the one-site operator O for the UMPS state
defined by the tensor A.
"""
function expect_local(A, O, l, r)
    l = tm_l_op(A, O, l)
    expectation = vec(l)'*vec(r)
    return expectation
end

expect_local

In [11]:
"""
    correlator_twopoint(A, O1, O2, m, l, r)

Return the (connected) two-point correlator of operators O1 and O2 for the
state UMPS(A), when O1 and O2 are i sites apart, where i ranges from 1 to m. In
other words, return <O1_0 O2_i> - <O1> <O2>, for all i = 1,...,m, where the
expectation values are with respect to the state |UMPS(A)>.
"""
function correlator_twopoint(A, O1, O2, m, l, r)
    local_O1 = expect_local(A, O1, l, r)
    local_O2 = expect_local(A, O2, l, r)
    disconnected = local_O1 * local_O2
    
    l = tm_l_op(A, O1, l)
    r = tm_r_op(A, O2, r)
    
    result = zeros(eltype(A), m)
    result[1] = vec(l)'*vec(r) - disconnected
    for i in 1:m
        r = tm_r(A, r)
        result[i] = vec(l)'*vec(r) - disconnected
    end
    return result
end

correlator_twopoint

In [12]:
"""
    correlation_length(A)

Return the correlation length ξ of the UMPS defined by A. ξ = - 1/ln(|lambda[2]|),
where lambda[2] is the eigenvalue of the MPS transfer matrix with second largest
magnitude. (We assume here that UMPS(A) is normalized.)
"""
function correlation_length(A)
    S, U = tm_eigs(A, "L", 2)
    s2 = S[2]
    ξ = -1/log(abs(s2))
    return ξ
end

correlation_length

# But I don't care about random MPSes!

Now that you know how to compute correlation functions for MPS states, we should start thinking about how do we get a useful MPS state in the first place. So far all our states have been defined by tensors $A$ with random elements.

Some fun little analytical examples exist, such as how to represent GHZ or W states as MPSes. You should google them up. The real question is though, given an arbitrary local Hamiltonian, can we find an MPS for the ground state. We know that for gapped Hamiltonians (i.e. most Hamiltonians) correlations in the ground state decay exponentially and entanglement is limited, so there's hope that the answer would be "yes we can".

You can think about this problem in a few different ways. One is to ask, what's the lowest energy state we can find on the manifold of MPS states. The question is then a matter of optimizing for minimal expectation value $\langle \psi | H | \psi \rangle$ under the constraint that the state $|\psi \rangle$ is an MPS. Another way to approach this would be writing down some expression for the exact ground state (an expression we obviously can't just go an evaluate, otherwise there would be no problem), and then try to massage that expression into an MPS.

Various algorithms for implementing these different approaches in different situations (finite-size, infinite-size, translation invariant or not, what kind of boundary conditions) exist. Some examples are the Density Matrix Renormalization Group (DMRG), Time-Dependent Variational Principle (TDVP) and Time-Evolving Block Decimation (TEBD). We'll implement an infinite-lattice version of TEBD. Why TEBD? Because it's simple, and because it you can also use it to do time-evolution of MPS states. (I maintain that the choice to pick TEBD is independent of the fact that Guifre invented it.)

Implementing iTEBD will be the rest of this minicourse. However, we'll only really get to that next time. First, we need to talk a bit about gauge freedom in MPS and how to fix this freedom, since TEBD will be making use of a specific gauge.

## Gauge freedom and canonical forms
In any tensor network, I can always take any contracted leg, and insert on that leg a pair of invertibale matrices $g g^{-1} = \mathbb{1}$. This doesn't affect the value the network contracts to in anyway: I've just changed the basis in which I perform one of the sums (traces).

This means that an MPS representation of a state is not unique. I can insert any $g g^{-1}$ on any of the virtual legs of the MPS, and the state stays the same. Obviously, I would like to stay within the framework of translation invariant MPSes, so I allow only for gauge transformations (as these changes of basis are called) of the form
```
--A-- -> --gi--A--g--
  |            |
```
where by `gi` I mean the inverse of $g$.

Since we have some freedom in picking $A$, we should probably try to make use of it somehow, while fixing the freedom. How? Well, we could try to impose some nice properties on $A$, if such a property can be achieved by a gauge transformation. And it turns out there's a really useful property that we can impose, which goes as follows.

First, we have to complicate our setup slightly. Previously our how MPS was defined by $A$. Now, let's think of an MPS that is instead written as
```
... --λ--Γ--λ--Γ--λ--Γ--λ-- ...
         |     |     |
```
Here $\Gamma$ is some tensor, and $\lambda$ is a diagonal matrix. Obviously we could just absorb $\lambda$ into $\Gamma$ and a get single tensor $A$. The point is, by using the gauge freedom, we can choose $\Gamma$ and $\lambda$ in such a way, that *$\lambda$ always has on its diagonal the Schmidt coefficients of partitioning the state into two parts at $\lambda$*. What are Schmidt coefficients? https://en.wikipedia.org/wiki/Schmidt_decomposition

Another way of stating the same condition is to say that the left dominant eigenvector of
```
--λ--Γ--
     |
--λ--Γ--
```
and the right dominant eigenvector of
```
--Γ--λ--
  |
--Γ--λ--
```
are both the identity matrix.

Choosing our gauge so that this property holds, turns out to be quite handy. You'll see next time how it's used in the TEBD algorithm, but you can probably already imagine that it's kinda nice that on each virtual legs you have these Schmidt coefficients $\lambda$, that immediately tell you everything you would want to know about the entanglement between the two sides of the state separated by this leg. The bond dimension is also obviously the Schmidt rank, which gives us a very concrete, physical characterization of what high bond dimension and low bond dimension mean.

So, given $A$, how do we find a gauge transformation such that the above property holds? I'm getting tired of typing, so just go read Section II of https://arxiv.org/pdf/0711.3960.pdf, that will explain it all. Then, it's time to define your homework.

## Homework 3
#### -1)
Make sure that, for the expectation values and correlation functions,<br>
a) Your expectation values for Hermitian operators are real.<br>
b) Your correlation functions decay like they should (make a plot).<br>
c) You can generate two-point correlators with $D=100$ for distances $1, \dots, 100$ or so in a few seconds.<br>
Feel free to make use of my code above, as long as you understand what's going on.

#### 0)
Read Section II of https://arxiv.org/pdf/0711.3960.pdf. Note that there they start from some $\Gamma$ and $\lambda$, that are not the canonical form, and try to turn them into $\Gamma'$ and $\lambda'$ that would be in the canonical form. We, on the other hand, start from just $A$. You can easily translate to their language by setting their $\Gamma$ to your $A$, and their $\lambda$ to just the identity matrix.

#### 1)
As explained in above reference, the form of MPS transfer matrix $T$, i.e. the symmetry it has because of how it consists of $A$ and $A^\ast$, guarantees that the dominant eigenvectors $l$ and $r$ are Hermitian and positive semi-definite. However, as we discussed in the last meeting, when we numerically ask for the eigenvectors, they come with an arbitrary phase. Last time we modified `normalize!` so that it partially fixes this phase (and also the norm of $l$ and $r$), imposing `vec(l)'*vec(r) == 1`. Your first job is to further modify `normalize!` so that it fully fixes the phases, so that the `l` and `r` it returns are Hermitian and positive semi-definite, in addition to having the property that `vec(l)'*vec(r) == 1`.

How do you do that? Well, you should definitely start by getting the `normalize!` function from above, and only make small modifications to that. Since we know that $r$ is actually $a \cdot \tilde{r}$, where $\tilde{r}$ is Hermitian and pos. semi-def. and $a$ is a scalar, we can just take the trace $\mathrm{Tr} r = a \mathrm{Tr} \tilde{r}$, and we know that $\mathrm{Tr} \tilde{r}$ is something real and positive. Any part of $\mathrm{Tr} r$ that is not real and positive, is $a$, and you divide $r$ by $a$ to get $\tilde{r}$. And similarly for $l$.

In [13]:
"""
    normalize!(A)

Normalize the UMPS defined by A, and return the dominant left and right
eigenvectors l and r of its transfer matrix, normalized so that they are
both Hermitian and positive semi-definite (when thought of as matrices),
and l'*r = 1.
"""
function normalize!(A)
    SR, UR, SL, UL = tm_eigs(A, "BOTH", 1)
    S1 = SR[1]
    A ./= sqrt(S1)
    
    l = UL[1]
    r = UR[1]  
    # We want both l and r to be Hermitian and pos. semi-def.
    # We know they are that, up to a phase.
    # We can find this phase, and divide it away, because it is also the
    # phase of the trace of l (respectively r).
    r_tr = trace(r)
    phase_r = r_tr/abs(r_tr)
    r ./= phase_r
    l_tr = trace(l)
    phase_l = l_tr/abs(l_tr)
    l ./= phase_l
    # Finally divide them by a real scalar that makes
    # their inner product be 1.
    n = vec(l)'*vec(r)
    abs_n = abs(n)
    phase_n = n/abs_n
    (phase_n ≉ 1) && warn("In normalize! phase_n = ", phase_n, " ≉ 1")
    sfac = sqrt(abs_n)
    l ./= sfac
    r ./= sfac
    return l, r
end

normalize!

#### 2)
Now that we can trust our $l$ and $r$ to be Hermitian and $\geq 0$, we can go ahead and implement a function that takes in $A$, $l$ and $r$, and outputs $\Gamma$ and $\lambda$, that define the same MPS, but now in canonical form.

In [24]:
"""
    canonical_form(A, l, r)

Return a three-valent tensor Γ and a vector λ, that define the canonical
of the UMPS defined by A. l and r should be the normalized dominant
left and right eigenvectors of A.
"""
function canonical_form(A, l, r)
    l_H = 0.5*(l + l')
    r_H = 0.5*(r + r')
    (l_H ≉ l) && warn("In canonical_form, l is not Hermitian: ", l)
    (r_H ≉ r) && warn("In canonical_form, r is not Hermitian: ", r)
    evl, Ul = eig(Hermitian(l_H))
    evr, Ur = eig(Hermitian(r_H))
    X = Ur * Diagonal(sqrt.(complex.(evr)))
    YT = Diagonal(sqrt.(complex.(evl))) * Ul'
    U, λ, V = svd(YT*X)
    Xi = Diagonal(sqrt.(complex.(1./evr))) * Ur'
    YTi = Ul * Diagonal(sqrt.(complex.(1 ./ evl)))
    @tensor Γ[x,i,y] := (V'[x,a] * Xi[a,b]) * A[b,i,c] * (YTi[c,d] * U[d,y])
    return Γ, λ
end

canonical_form

#### 3)
Prove to yourself that<br>
a)<br>
the way you've chosen to transform the MPS, going form $A$ to $(\Gamma, \lambda)$, is actually a gauge transformation. In other words, prove that they both define the same state.

b)<br>
that the MPS defined $(\Gamma, \lambda)$ is actually in canonical form, i.e., that the left dominant eigenvector of
```
--λ--Γ--
     |
--λ--Γ--
```
and the right dominant eigenvector of
```
--Γ--λ--
  |
--Γ--λ--
```
are both the identity matrix. Do this both numerically and analytically.

In [18]:
let
    d = 2
    D = 20
    A = rand_UMPS(d, D; keep_it_real=false)
    
    # Test the new normalization function.
    l, r = normalize!(A)
    SR, UR, SL, UL = tm_eigs(A, "BOTH", 1)
    # The MPS is normalized.
    println(SR[1] ≈ 1.0)
    println(SL[1] ≈ 1.0)
    # l and r are really eigenvectors, and have their inner product be 1.
    println(l ≈ tm_l(A, l))
    println(r ≈ tm_r(A, r))
    println(vec(l)'*vec(r) ≈ 1.0)
    # l and r are really Hermitian.
    println(vecnorm(l - l'))
    println(vecnorm(r - r'))
    # l and r are really pos. semi-def.
    evl = eig(l)[1]
    evr = eig(r)[1]
    allposl = all(abs(imag(i)) < 1e-14 && real(i) >= 0 for i in evl)
    allposr = all(abs(imag(i)) < 1e-14 && real(i) >= 0 for i in evr)
    println(allposl)
    println(allposr)
end

true
true
true
true
true
2.6478124784463845e-15
3.355021370262196e-15
true
true


In [23]:
let
    d = 2
    D = 10
    A = rand_UMPS(d, D; keep_it_real=false)
    
    l, r = normalize!(A)
    Γ, λ = canonical_form(A, l, r)
    λm = diagm(λ)
    
    # Check that identity really is the dominant eigenvector as it should.
    @tensor should_be_unity_l[x,y] := (λm[a,b] * Γ[b,i,x]) * (conj(λm)[a,c] * conj(Γ)[c,i,y])
    @tensor should_be_unity_r[x,y] := (Γ[x,i,b] * λm[b,a]) * (conj(Γ)[y,i,c] * conj(λm)[c,a])
    @show vecnorm(should_be_unity_l - eye(D,D))
    @show vecnorm(should_be_unity_r - eye(D,D))
    
    # Check that we get the same two-point correlators for the original A,
    # and for a couple of different ways of contracting together λ and Γ.
    O1 = randn(d, d)
    O2 = randn(d, d)
    O1 = (O1 + O1')/2
    O2 = (O2 + O2')/2
    
    m = 3
    # The original A.
    corrsA = correlator_twopoint(A, O1, O2, m, l, r)
    # Contract λ into Γ from the right.
    @tensor A2[x,i,y] := Γ[x,i,a] * λm[a,y]
    l2, r2 = normalize!(A2)
    corrsA2 = correlator_twopoint(A2, O1, O2, m, l2, r2)
    println(all(corrsA .≈ corrsA2))
    # Contract λ into Γ from the left.
    @tensor A3[x,i,y] := λm[x,a] * Γ[a,i,y]
    l3, r3 = normalize!(A3)
    corrsA3 = correlator_twopoint(A3, O1, O2, m, l3, r3)
    println(all(corrsA .≈ corrsA3))
end

vecnorm(should_be_unity_l - eye(D, D)) = 1.9894087737535425e-14
vecnorm(should_be_unity_r - eye(D, D)) = 2.3205209933034125e-14
true
true


## iTEBD

Finally, we have arrived at a point where we have everything we need to implement iTEBD, and use it to find the ground state for a given Hamiltonian. We'll first go through the Suzuki-Trotter decomposition of the imaginary time-development operator, see how that leads to an algorithm (iTEBD) for finding an MPS for the ground state, and then finally see why we should keep our MPS in the canonical form while applying this algorithm.

Given a Hamiltonian $H$, here's one way to define the ground state:
$$ |E_0\rangle \sim e^{-\beta H} | \psi \rangle, \quad \beta >> 1. $$
Here $\sim$ means $=$ up to normalization. $| \psi \rangle$ can be any state, as long as it's not orthogonal to $|E_0 \rangle$ (just pick one at random). If $\beta$ is large enough, $e^{\beta H}$ becomes proportional to the projector to $| E_0 \rangle$, and the above equation holds.

Of course, $e^{- \beta H}$ is a matrix exponentially big in the system size. Instead of trying to evaluate it, what we'll do is decompose it into a circuit of local gates. This is called the Suzuki-Trotter decomposition.

For this, we'll need to assume $H$ is a sum over local terms. Let's assume it consists of two-site terms:
$$ H = \sum_i h_{i,i+1}. $$
Then we can write 
$$ H = \sum_{i \text{ odd}} h_{i,i+1} + \sum_{i \text{ even}} h_{i,i+1} = H_{\text{odd}} + H_{\text{even}}. $$
The useful point about this is that all the terms in $H_{\text{odd}}$ operate on different sites, and thus commute with each other, and similarly for $H_{\text{even}}$. Next, introduce a small parameter $\tau$, and use the Baker–Campbell–Hausdorff formula as follows:
$$ e^{-\beta H} = \big[ e^{-\tau H} \big]^{\frac{\beta}{\tau}}
= \big[ e^{-\tau H_{\text{odd}} \; -\tau H_{\text{even}}} \big]^{\frac{\beta}{\tau}}
= \big[ e^{-\tau H_{\text{odd}}} \; e^{-\tau  H_{\text{even}}} \; e^{-\tau^2 (\text{commutators of $H_{\text{odd}}$ and $H_{\text{even}}$)}} \big]^{\frac{\beta}{\tau}}.$$
Making $\tau$ small enough so that anything $O(\tau^2)$ can be ignored, makes this
$$ \big[ \prod_{i \text{ odd}} e^{-\tau h_{i, i+1}} \; \prod_{i \text{ even}} e^{-\tau h_{i, i+1}} \big]^{\frac{\beta}{\tau}}.$$
Diagrammatically we can write this as

<img src="fig/trotter.svg">

Based on this, we can formulate the basic idea of the iTEBD algorithm (note that the i in iTEBD stands for infinite systems, but so far everything we've done works the same way for finite systems): Take the expression $e^{-\beta H} | \psi \rangle$. Decompose $e^{-\beta H}$ in to circuit using the above Suzuki-Trotter decomposition, with some choice of the parameter $\tau$ that is small. Take the initial state $| \psi \rangle$ to be an MPS. The expression for the ground state then looks like

<img src="fig/trotter_gs.svg">

With these steps, we've gotten rid of all the exponentially large objects in $e^{-\beta H} | \psi \rangle$: The evolution operator is described by local gates, and the state is defined by local MPS tensors. We can then absorb the layers of the circuit into the MPS, layer by layer, contracting only local things at a time. If we assume translation invariance this can all be done at the thermodynamic limit, i.e., with an infinite system. By "absorbing" a layer I mean this:

<img src="fig/trotter_absorb.svg">

At the last step the four-valent tensor has been split into two MPS matrices. This can be done by considering it as a matrix from the two left-most legs to the two right-most legs, and performing and a singular value decomposition (SVD).

With this method we can keep absorbing layers of the circuit $e^{-\beta H}$ into our MPS, effectively increasing $\beta$, until the state doesn't change significantly any more, i.e., until the procedure converges. At the end, our MPS should be the ground state of the Hamiltonian, subject to the caveat that since our $\tau$ is small, and not exactly zero, and our $\beta$ is large, and not exactly infinite, we only find the ground state approximately (the error coming from ignoring the $\tau^2$ terms is called the Trotter error).

Seems great! So why don't we get coding? Well, there's one problem here. Every time we use an SVD to split

<img src="fig/trotter_split.svg">

we are SVDing a $dD \times dD$ matrix. This means there are $dD$ singular values, meaning the bond dimension of our MPS after this procedure will be $dD$ (whereas a moment before it was just $D$). So our bond dimension will grow during this procedure, and exponentially too! This is not good.

However, we never claimed we wanted the exact ground state. What we wanted was a low bond dimension MPS that approximates the ground state. So somehow we should reduce the bond dimension back to what it was, while not changing the state our MPS describes too much (or as little as possible). We should probably take our MPS tensors, that now have some indices ranging over $dD$ values, and throw out some of the elements (like throwing out some columns of a matrix) so that all the dimensions are at most $D$. But which ones do we throw out?

Turns out, you should always throw out the parts that correspond to the smallest Schmidt values. This is the same result as the fact that the optimal low-rank approximation to matrix is given by its truncated SVD (see Low rank approximation in https://en.wikipedia.org/wiki/Singular-value_decomposition#Applications_of_the_SVD), since the Schmidt decomposition is just an SVD in disguise. Thus we should be performing this whole iTEBD procedure in the canonical form, so that on each leg we have the Schmidt value corresponding to that bipartition of the state. Then every time we perform one of the SVDs that increases the bond dimension, we can go back to the canonical form and remove from the neighbouring tensors the parts that correspond to the smallest Schmidt values (which has the same effect as setting those values to zero).

Thus a naive summary of the iTEBD algorithm would be as follows:
1. Absorb a layer of the circuit for $e^{-\beta H}$ into your MPS.
2. Use SVDs to transform the network back into an MPS, but now with a larger bond dimension.
3. Gauge transform this network into the canonical form.
4. Throw out the smallest Schmidt values on some of the legs, to reduce the bond dimension back to $D$.
5. Go to 1., unless your state has already converged, i.e., isn't changing any more, in which case we are done.

Note that every second time we go through this process, we should absorb a layer of $e^{-\tau H_{\text{odd}}}$ and every second time a layer of $e^{-\tau H_{\text{even}}}$.

I call the above summary "naive", because I've ignored some details. For instance, the canonical form of your MPS will now have translation symmetry only by two sites, which means you'll have to keep to tensors $\Gamma_A$ and $\Gamma_B$ and corresponding two vectors of Schmidt values $\lambda_A$ and $\lambda_B$. It also turns out it's not important to be canonicalizing your MPS at every single step: If you start with an MPS in canonical form, you can perform the absorption, SVD and truncation all with recanonicalizing in between, since your MPS will stay close enough to being canonical. You can even repeat this for a few iterations before having to canonicalize again. These kinds of details you should check up on your own from the literature. I mainly recommend<br>
https://arxiv.org/pdf/cond-mat/0605597.pdf<br>
for understanding the iTEBD algorithm, and Section II of<br>
https://arxiv.org/pdf/0711.3960.pdf<br>
for how to deal with the canonical form (including the one with two-site translation symmetry).
Other resources exist, which you can try if you want to, especially given that the first paper above is quite terse. There's even a wikipedia article, https://en.wikipedia.org/wiki/Time-evolving_block_decimation, although I must admit I haven't read it.

Your homework for next time is to go and implement this! You should start by reading https://arxiv.org/pdf/cond-mat/0605597.pdf, and then try putting all of it into code. Don't worry if you don't get it all done by the next meeting. What I want to see is a serious attempt to get as far as you can (including writing some code!). Once you hit a roadblock and don't know what to do anymore, you should try to understand what exactly it is you don't understand, and then come to the next meeting with questions. Next time we'll go through what you got done and clarify what needs to be clarified. We'll hopefully also talk a bit about things like performance, how to measure convergence, etc.

I've written below some templates that you can use as a basis of your own code. They also include some hints as to how to go about implementing things.

In [31]:
# U should be the four-valent tensor e^(-\tau h).
# D is the bond dimension to be used.
# You probably want to add some other arguments for things like
# how exactly to decide when convergence has been reached.
function itebd_optimize(U, D)
    d = size(U, 1)
    ΓA, λA, ΓB, λB = itebd_random_initial(d, D)
    # Here there should be a loop, that applies one step of iTEBD,
    # i.e., absorbs U into ΓA, λA, ΓB, λB, and splits with an SVD
    # to get back to an MPS defined by some other ΓA2, λA2, ΓB2, λB2,
    # and truncates the SVD back to bond dimension D.
    # Your ΓA2, λA2, ΓB2, λB2 will not be exactly in the canonical form
    # any more after all this, so you should probably recanonicalize.
    #
    # Finally, you should check whether doing all this changed your MPS
    # significantly: If not, the iTEBD procedure has converged, and we can
    # quit and return. Convergence can be checked in many ways, but one
    # good one is to compare the Schmidt values λA2, λB2 to the previous
    # Schmidt values λA, λB, and see if they've changed a lot.
    return ΓA, λA, ΓB, λB
end

itebd_optimize (generic function with 1 method)

In [30]:
function itebd_step(ΓA, λA, ΓB, λB, U)
    # This is the heart of the algorithm: Absorb, split, truncate.
    # See Figure 3 of https://arxiv.org/pdf/cond-mat/0605597.pdf
    # of what should go in here. Remember that you should do the same
    # thing twice, first U operating of odd pairs of sites and then on
    # even pairs of sites.
end

itebd_step (generic function with 1 method)

In [29]:
function itebd_random_initial(d, D)
    # Generate a random initial guess for the MPS. It should have
    # translation symmetry by two sites, i.e., it should consist of
    # tensors ΓA, λA, ΓB, λB, and be in the canonical form.
end

itebd_random_initial (generic function with 1 method)

In [28]:
# A handy function for truncating SVDs. You'll need this in implementing itebd_step.
"""
    truncate_svd(U, S, V, D)

Given an SVD of some matrix M as M = U*diagm(S)*V', truncate this
SVD, keeping only the D largest singular values.
"""
function truncate_svd(U, S, V, D)
    U = U[:, 1:D]
    S = S[1:D]
    V = V[:, 1:D]
    return U, S, V
end

truncate_svd

In [27]:
function double_canonicalize(ΓA, λA, ΓB, λB)
    # Take in an MPS defined by ΓA, λA, ΓB, λB, and do a gauge transformation to
    # put it in a canonical form. See the end of Section II of
    # https://arxiv.org/pdf/0711.3960.pdf for how to do this.
end

double_canonicalize (generic function with 1 method)

## Let's test this algorithm of ours!

In [32]:
function build_ising_ham(h=1.0)
    X = [0 1; 1 0]
    Z = [1 0; 0 -1]
    I2 = eye(2)
    XX = kron(X, X)
    ZI = kron(Z, I2)
    IZ = kron(I2, Z)
    H = -(XX + h/2*(ZI+IZ))
    return H
end

build_ising_ham (generic function with 2 methods)

In [43]:
let
    # Here's a piece that you can use as a basis for testing your iTEBD.
    # You should probably implement function that measures the per site ground
    # state energy (very much like your previous expect_local, but now for an
#)
    τ = 1e-2
    D = 50
    h = build_ising_ham()
    U = expm(-τ*h)
    U = reshape(U, (2,2,2,2))
    h = reshape(h, (2,2,2,2))
    ΓA, λA, ΓB, λB = itebd_optimize(U, D)
end

In iTEBD, eps = 1.863e+00, counter = 1, energy = -1.255681618864e+00, off by 1.379e-02
In iTEBD, eps = 1.446e-01, counter = 2, energy = -1.267309492347e+00, off by 4.657e-03
In iTEBD, eps = 4.229e-02, counter = 3, energy = -1.270053100200e+00, off by 2.503e-03
In iTEBD, eps = 3.387e-02, counter = 4, energy = -1.271302506296e+00, off by 1.521e-03
In iTEBD, eps = 2.773e-02, counter = 5, energy = -1.271953216456e+00, off by 1.010e-03
In iTEBD, eps = 2.311e-02, counter = 6, energy = -1.272328045972e+00, off by 7.159e-04
In iTEBD, eps = 1.971e-02, counter = 7, energy = -1.272561340333e+00, off by 5.327e-04
In iTEBD, eps = 1.710e-02, counter = 8, energy = -1.272715532954e+00, off by 4.116e-04
In iTEBD, eps = 1.506e-02, counter = 9, energy = -1.272822393042e+00, off by 3.276e-04
In iTEBD, eps = 1.343e-02, counter = 10, energy = -1.272899329676e+00, off by 2.672e-04
In iTEBD, eps = 1.210e-02, counter = 11, energy = -1.272956479819e+00, off by 2.223e-04
In iTEBD, eps = 1.100e-02, counter = 12, 

(Complex{Float64}[-0.944434-2.40795e-12im -0.373705-2.34097e-12im; 0.174414+0.0692808im -0.650334-0.258326im; … ; -3.58175e-5+9.56548e-5im -4.80892e-5-4.74559e-5im; -2.56068e-5-5.11531e-7im 5.42919e-5+4.88512e-6im]

Complex{Float64}[0.18343-7.62295e-12im -0.695067+1.87727e-11im; -2.42888-0.9648im 1.21576+0.482925im; … ; 0.00083375+0.000408465im 0.00138506-0.000467533im; -0.000233485-0.000176437im 8.58347e-5-0.000192472im]

Complex{Float64}[-0.246986+1.52566e-11im 0.545309-1.83567e-11im; -1.78995-0.711005im 0.43749+0.17378im; … ; -0.00441753-0.00253672im -0.00571466+0.00872578im; 0.00678015-0.00381446im 0.00234685+0.000280382im]

...

Complex{Float64}[7.14371e-5+3.19402e-5im -1.82364e-5-1.28023e-5im; -0.000323343-0.000327746im -0.000817066-0.000809968im; … ; 8.69253e7-7.67621e7im 5.68044e7+2.58622e7im; 1.53577e7-1.04811e7im 4.789e7-1.66212e7im]

Complex{Float64}[1.99134e-5-1.4998e-5im -1.50992e-5+1.67924e-5im; 6.25346e-5-0.000161775im -0.000216057-2.52747e-5im; … ; 2.42598e8+4.98808e7im

In [118]:
# τ = 1e-2, D = 70
# In iTEBD, eps = 2.824e-04, counter = 467, energy = -1.272827043163e+00, off-by 3.240e-04
# τ = 1e-2, D = 100
# In iTEBD, eps = 1.863e-04, counter = 500, energy = -1.273042308731e+00, off by 1.549e-04

In [38]:
# In iTEBD, eps = 1.019e-02, counter = 50, energy = -1.154290569054e+00, off by 9.342e-02
# 55.273693 seconds (1.53 M allocations: 9.625 GiB, 3.11% gc time)

# After vectorizing canonicalize_double:
# In iTEBD, eps = 1.993e-02, counter = 50, energy = -1.165382092223e+00, off by 8.471e-02
# 64.933059 seconds (2.02 M allocations: 10.980 GiB, 3.35% gc time)

# After further making multiplications in-place in canonicalize_double
# In iTEBD, eps = 2.905e-02, counter = 50, energy = -1.227661648993e+00, off by 3.580e-02
# 29.377446 seconds (4.06 M allocations: 10.809 GiB, 5.31% gc time)

# After vectorizing and making things happen in place in itebd_halfstep:
# In iTEBD, eps = 9.356e-03, counter = 50, energy = -1.245828616447e+00, off by 2.153e-02
# 31.969622 seconds (5.32 M allocations: 12.147 GiB, 6.14% gc time)